# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = '../output_data/city_data.csv'
cities_df = pd.read_csv(file)

cities_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity %,Cloudiness,Wind Speed
0,0,Ponta do Sol,PT,32.6667,-17.1000,20.29,94,97,5.98
1,1,Richards Bay,ZA,-28.7830,32.0377,26.26,61,73,4.83
2,2,Cape Town,ZA,-33.9258,18.4232,32.01,39,0,8.23
3,3,Kendari,ID,-3.9450,122.4989,26.34,87,92,0.47
4,4,Inzer,RU,54.2167,57.5556,-2.36,93,100,4.74


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Latitude", "Longitude"]]
humid = cities_df["Humidity %"].astype(float)


fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
cities_df.dropna(inplace = True)
hotel_df = cities_df.loc[(cities_df['Temperature']<27) & (cities_df['Temperature']>21)
                             &(cities_df['Cloudiness']==0)
                             &(cities_df['Wind Speed']<10),:].reset_index().copy()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
hotel_df['Hotel Name']= "" 
for x in range(len(hotel_df)):
    lng = hotel_df['Longitude'][x]
    lat = hotel_df['Latitude'][x]
    response = requests.get(f'{base_url}?key={g_key}&location={lat},{lng}&radius=5000&type=lodging').json()
    try:
        hotel_df['Hotel Name'][x]= response['results'][14]['name']
    
    except IndexError:
        hotel_df['Hotel Name'][x] = 'Not Found'

<ipython-input-5-a7459100a756>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][x]= response['results'][14]['name']
<ipython-input-5-a7459100a756>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][x] = 'Not Found'


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [7]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig1.add_layer(markers)
# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))